In [1]:
# install experiment widget
!jupyter nbextension install --py --user azureml.train.widgets

# enable experiment widget
!jupyter nbextension enable --py --user azureml.train.widgets

Installing /anaconda/envs/amlsdkenv/lib/python3.6/site-packages/azureml/train/widgets/static -> azureml_train_widgets
Up to date: /home/chenhui/.local/share/jupyter/nbextensions/azureml_train_widgets/index.js
Up to date: /home/chenhui/.local/share/jupyter/nbextensions/azureml_train_widgets/extension.js
Up to date: /home/chenhui/.local/share/jupyter/nbextensions/azureml_train_widgets/packages/labextension/azureml_train_widgets-1.0.0.tgz
- Validating: OK

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable azureml.train.widgets --user --py
    
Enabling notebook extension azureml_train_widgets/extension...
      - Validating: OK


In [2]:
%matplotlib inline
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt

In [3]:
import azureml
from azureml.core import Workspace, Run

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  0.1.74


In [4]:
from azureml.telemetry import set_diagnostics_collection
set_diagnostics_collection(send_diagnostics=True)

In [6]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Found the config file in: /home/chenhui/TSPerf/retail_sales/OrangeJuice_Pt_3Weeks_Weekly/submissions/DilatedCNN/config.json
Workspace name: chhws
Azure region: southcentralus
Subscription id: ff18d7a8-962a-406c-858f-49acd23d6c01
Resource group: tsperf


## Create an Azure ML experiment

In [7]:
from azureml.core import Experiment

#script_folder = './tf-mnist'
#os.makedirs(script_folder, exist_ok=True)

exp = Experiment(workspace=ws, name='tune_dcnn')

## Upload Orange Juice dataset to default datastore

In [9]:
ds = ws.get_default_datastore()
ds.upload(src_dir='../../data', target_path='data', overwrite=True, show_progress=True)

$AZUREML_DATAREFERENCE_1094968f03694512b46340b93f7d2c59

## Create Batch AI cluster as compute target

In [11]:
from azureml.core.compute import ComputeTarget, BatchAiCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpucluster"

try:
    # look for the existing cluster by name
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    if type(compute_target) is BatchAiCompute:
        print('Found existing compute target {}.'.format(cluster_name))
    else:
        print('{} exists but it is not a Batch AI cluster. Please choose a different name.'.format(cluster_name))
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = BatchAiCompute.provisioning_configuration(vm_size="STANDARD_NC6", # GPU-based VM
                                                                #vm_priority='lowpriority', # optional
                                                                autoscale_enabled=True,
                                                                cluster_min_nodes=0, 
                                                                cluster_max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
    # Use the 'status' property to get a detailed status for the current cluster. 
    print(compute_target.status.serialize())

Creating a new compute target...
Creating
succeeded
BatchAI wait for completion finished
Minimum number of nodes requested have been provisioned
{'allocationState': 'steady', 'allocationStateTransitionTime': '2018-11-11T02:53:26.585000+00:00', 'creationTime': '2018-11-11T02:53:17.872000+00:00', 'currentNodeCount': 0, 'errors': None, 'nodeStateCounts': {'idleNodeCount': 0, 'leavingNodeCount': 0, 'preparingNodeCount': 0, 'runningNodeCount': 0, 'unusableNodeCount': 0}, 'provisioningState': 'succeeded', 'provisioningStateTransitionTime': '2018-11-11T02:53:25.738000+00:00', 'scaleSettings': {'manual': None, 'autoScale': {'maximumNodeCount': 4, 'minimumNodeCount': 0, 'initialNodeCount': 0}}, 'vmPriority': 'dedicated', 'vmSize': 'STANDARD_NC6'}


In [12]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

gpucluster BatchAI Succeeded


## Copy the training files into the script folder

In [14]:
from azureml.train.dnn import TensorFlow

In [ ]:
script_folder = './tune-dcnn'
os.makedirs(script_folder, exist_ok=True)